In [ ]:
import os
import cv2
import glob
import random
import numpy as np

from tensorflow.keras import optimizers
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from tensorflow.keras.models import load_model, Model, model_from_json, Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, InputLayer, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras.layers import Dropout, Flatten, GlobalAveragePooling2D

from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, LabelEncoder

from crossdata_metrics import metrics, data_balanced, train_crossdataset



# Model

In [ ]:
mobile = MobileNet(include_top=False, weights='imagenet', input_shape=(224,224,3))
for layer in mobile.layers:
    layer.trainable = False
output = mobile.layers[-1].output
output = Flatten()(output)
mobile_model = Model(mobile.input, output)

# Load dataset

In [ ]:
def load_path(path):

    labels_ = os.listdir(path)

    features = []
    labels   = []

    for i, label in enumerate(labels_):
        cur_path = path + "/" + label 

        for image_path in glob.glob(cur_path + "/*"):

            img = image.load_img(image_path, target_size=(224, 224))
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            feature = mobile_model.predict(x)
            flat = feature.flatten()
            
            features.append(flat)
            labels.append(label)
            
        print ("[INFO] completed label - " + label)

    return features, labels



In [ ]:
### ACRIMA
ACRIMA_data, ACRIMA_labels = load_path("D:/Glaucoma/Banco_de_dados/ACRIMA/Images")

## REFUGE
REFUGE_data_train, REFUGE_labels_train = load_path("D:/Glaucoma/Banco_de_dados/REFUGE/cortes2")
REFUGE_data_test, REFUGE_labels_test = load_path("D:/Glaucoma/Banco_de_dados/REFUGE/REFUGE-Test400/Cortes_test")

#### RIM-ONE 
RO_data_train, RO_labels_train = load_path("D:/Glaucoma/Banco_de_dados/RIM-ONE_DL_images/partitioned_randomly/training_set")
RO_data_test, RO_labels_test = load_path("D:/Glaucoma/Banco_de_dados/RIM-ONE_DL_images/partitioned_randomly/test_set")


In [ ]:
%%time
# Refuge + Acrima + Rim-One DL
data = np.concatenate([ACRIMA_data, REFUGE_data_train, REFUGE_data_test, RO_data_train,RO_data_test], axis = 0)
labels_ = np.concatenate([ACRIMA_labels, REFUGE_labels_train, REFUGE_labels_test, RO_labels_train, RO_labels_test], axis = 0)

# Refuge + Acrima
data_re_ac = np.concatenate([ACRIMA_data, REFUGE_data_train, REFUGE_data_test], axis = 0)
labels_re_ac_ = np.concatenate([ACRIMA_labels, REFUGE_labels_train, REFUGE_labels_test], axis = 0)

# Rim-One + Acrima
data_ro_ac = np.concatenate([ACRIMA_data, RO_data_train,RO_data_test], axis = 0)
labels_ro_ac_ = np.concatenate([ACRIMA_labels, RO_labels_train, RO_labels_test], axis = 0)

# Rim-One + Refuge
data_ro_re = np.concatenate([REFUGE_data_train, REFUGE_data_test, RO_data_train,RO_data_test], axis = 0)
labels_ro_re_ = np.concatenate([REFUGE_labels_train, REFUGE_labels_test, RO_labels_train, RO_labels_test], axis = 0)

# Refuge
data_re = np.concatenate([REFUGE_data_train, REFUGE_data_test], axis = 0)
labels_re_ = np.concatenate([REFUGE_labels_train, REFUGE_labels_test], axis = 0)

# Acrima
data_ac = np.concatenate([ACRIMA_data], axis = 0)
labels_ac_ = np.concatenate([ACRIMA_labels], axis = 0)

# Rim-One
data_ro = np.concatenate([RO_data_train,RO_data_test], axis = 0)
labels_ro_ = np.concatenate([RO_labels_train, RO_labels_test], axis = 0)


le = LabelEncoder()
labels = le.fit_transform(labels_)
labels_re_ac = le.fit_transform(labels_re_ac_)
labels_ro_ac = le.fit_transform(labels_ro_ac_)
labels_ro_re = le.fit_transform(labels_ro_re_)
labels_re = le.fit_transform(labels_re_)
labels_ac = le.fit_transform(labels_ac_)
labels_ro = le.fit_transform(labels_ro_)

print("Data: ", np.shape(data))
print("Normal: ", len(data[labels==0]))
print("Glaucoma: ", len(data[labels==1]))


# Classification

In [ ]:
svm = SVC(kernel='rbf', C=3, probability=True)
mlp = MLPClassifier(activation='relu',max_iter=500,alpha=0.01,hidden_layer_sizes=(50,50))
xgb = XGBClassifier(colsample_bytree=0.8, max_depth=6, n_estimators= 300, subsample= 0.4, seed=0)


models = []
# models.append(('SVM', SVC(C=3, kernel='rbf')))
# models.append(('XGB', XGBClassifier(colsample_bytree=0.8, max_depth=6, n_estimators= 300, subsample= 0.4, seed=0))) 
# models.append(('MLP', MLPClassifier(activation='relu',max_iter=500,alpha=0.01,hidden_layer_sizes=(50,50))))
models.append(('VOT', VotingClassifier(estimators = [('svm', svm),('xgb', xgb),('mlp', mlp)], voting='hard')))



# Results

In [ ]:
# # Desbalanceado
def results_unbalanced():
    train_crossdataset(data_ro_ac, labels_ro_ac, data_re, labels_re, 'None', models, 'mobile')
    train_crossdataset(data_ro_re, labels_ro_re, data_ac, labels_ac, 'None', models, 'mobile')
    train_crossdataset(data_re_ac, labels_re_ac, data_ro, labels_ro, 'None', models, 'mobile')
    return 

results_unbalanced()

In [ ]:
# Balanceamento aleatório
def results_balanced_random():

    train_crossdataset(data_ro_ac, labels_ro_ac, data_re, labels_re, 'random', models, 'mobile')
    train_crossdataset(data_ro_re, labels_ro_re, daata_ac, labels_ac, 'random', models, 'mobile')
    train_crossdataset(data_re_ac, labels_re_ac, data_ro, labels_ro, 'random', models, 'mobile')
    
    return 
    
results_balanced_random()

In [ ]:
# Balanceamento Near Miss
def results_balanced_nm():
    train_crossdataset(data_ro_ac, labels_ro_ac, data_re, labels_re, 'nm', models, 'mobile')
    train_crossdataset(data_ro_re, labels_ro_re, data_ac, labels_ac, 'nm', models, 'mobile')
    train_crossdataset(data_re_ac, labels_re_ac, data_ro, labels_ro, 'nm', models, 'mobile')
    return 
    
results_balanced_nm()

In [ ]:
# Balanceamento cluster

def results_balanced_cluster():
    train_crossdataset(data_ro_ac, labels_ro_ac, data_re, labels_re, 'cluster', models, 'mobile')
    train_crossdataset(data_ro_re, labels_ro_re, data_ac, labels_ac, 'cluster', models, 'mobile')
    train_crossdataset(data_re_ac, labels_re_ac, data_ro, labels_ro, 'cluster', models, 'mobile')
    
    return 
    
results_balanced_cluster()